In [1]:
!pip install gym pyvirtualdisplay > /dev/null 2>&1
!apt-get install -y xvfb python-opengl ffmpeg > /dev/null 2>&1
!apt-get update > /dev/null 2>&1
!apt-get install cmake > /dev/null 2>&1
!pip install --upgrade setuptools 2>&1
!pip install ez_setup > /dev/null 2>&1
!pip install gym[atari] > /dev/null 2>&1
!pip install git+https://github.com/tensorflow/docs > /dev/null 2>&1
!pip install gym[classic_control]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 14.9 MB/s eta 0:00:00
  Attempting uninstall: setuptools
    Found existing installation: setuptools 67.6.0
    Uninstalling setuptools-67.6.0:
      Successfully uninstalled setuptools-67.6.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.9.0 requires jedi>=0.10, which is not installed.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 64.0 MB/s eta 0:00:00
  Attempting uninstall: pygame
    Found existing installation: pygame 2.3.0
    Uninstalling pygame-2.3.0:
      Successfully uninstalled pygame-2.3.0


In [2]:
import numpy as np
import random
import torch
import torch.nn as nn  
import torch.nn.functional as F
from collections import namedtuple, deque
import torch.optim as optim
import datetime
import gym
from gym.wrappers.record_video import RecordVideo
import glob
import io
import base64
import matplotlib.pyplot as plt
from IPython.display import HTML
from pyvirtualdisplay import Display
import tensorflow as tf
from IPython import display as ipythondisplay
from PIL import Image
import tensorflow_probability as tfp

In [3]:
env = gym.make('CartPole-v1')
env.seed(0)

state_shape = env.observation_space.shape[0]
no_of_actions = env.action_space.n

print(state_shape)
print(no_of_actions)
print(env.action_space.sample())
print("----")

state = env.reset()   

print(state)
print("----")

action = env.action_space.sample()  

print(action)
print("----")

next_state, reward, done, info = env.step(action) 

print(next_state)
print(reward)
print(done)
print(info)
print("----")

4
2
1
----
[ 0.01369617 -0.02302133 -0.04590265 -0.04834723]
----
1
----
[ 0.01323574  0.17272775 -0.04686959 -0.3551522 ]
1.0
False
{}
----


/usr/local/lib/python3.9/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.9/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.9/dist-packages/gym/core.py:256: DeprecationWarning: WARN: Function `env.seed(seed)` is marked as deprecated and will be removed in the future. Please use `env.reset(seed=seed)` instead.
  deprecation(


In [18]:
import gym
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

class ActorCritic(nn.Module):

    def __init__(self, state_dim, action_dim, hidden1, hidden2):
        super(ActorCritic, self).__init__()

        self.actor = nn.Sequential(
            nn.Linear(state_dim, hidden1),
            nn.ReLU(),
            nn.Linear(hidden1, hidden2),
            nn.ReLU(),
            nn.Linear(hidden2, action_dim),
            nn.Softmax(dim=-1),
        )
        self.critic = nn.Sequential(
            nn.Linear(state_dim, hidden1),
            nn.ReLU(),
            nn.Linear(hidden1, hidden2),
            nn.ReLU(),
            nn.Linear(hidden2, 1)
        )

    def forward(self, state):
        action_probs = self.actor(state)
        value = self.critic(state)
        return action_probs, value

In [23]:
HIDDEN1 = 128
HIDDEN2 = 64
GAMMA = 0.99
LR = 5e-4
SEED = 101

In [20]:
class Agent():

  def __init__(self, state_dim, action_dim, gamma = GAMMA, seed = SEED):
    super(Agent, self).__init__()
    self.gamma = gamma
    self.ac_model = ActorCritic(state_dim, action_dim, HIDDEN1, HIDDEN2)
    self.optimizer = optim.Adam(self.ac_model.parameters(), lr = LR)
    np.random.seed(seed)

  def sample_action(self, state):
    state_tensor = torch.FloatTensor(state).unsqueeze(0)
    action_probs, _ = self.ac_model(state_tensor)
    action = np.random.choice(env.action_space.n, p=action_probs.detach().numpy()[0])
    return action

  def actor_loss(self, action_probs, action, advantage):
    actor_loss = -torch.log(action_probs[0, action]) * advantage.detach()
    return actor_loss

  def critic_loss(self, advantage):
    return advantage.pow(2)

  def learn(self, states, actions, rewards, state, action, reward):

    for t in range(len(states)):
      G = 0
      for k, r in enumerate(rewards[t:]):
          G += self.gamma ** k * r

      state_i = torch.FloatTensor(states[t]).unsqueeze(0)
      _, value_i = self.ac_model(state_i)
      advantage = G - value_i

      action_i_probs, _ = self.ac_model(state_i)
      actor_loss = -torch.log(action_i_probs[0, actions[t]]) * advantage.detach()
      critic_loss = advantage.pow(2)
      total_loss = actor_loss + critic_loss

      self.optimizer.zero_grad()
      total_loss.backward()
      self.optimizer.step()

In [ ]:
total_rewards = []
n_episodes = []

for i in range(10):
  env = gym.make('Acrobot-v1')
  env._max_episode_steps = 500
  env.seed(SEED)
  state_dim = env.observation_space.shape[0]
  action_dim = env.action_space.n
  reward_list = []
  average_reward_list = []
  begin_time = datetime.datetime.now()

  agent = Agent(state_dim, action_dim)
  num_episodes = 10000

  for episode in range(1, num_episodes+1):
    state = env.reset()
    done = False
    episode_reward = 0
    states, actions, rewards = [], [], []
    while not done:
      action = agent.sample_action(state)
      next_state, reward, done, info = env.step(action)
      states.append(state)
      actions.append(action)
      rewards.append(reward)
      episode_reward += reward
      state = next_state
    agent.learn(states, actions, rewards, state, action, reward)
    reward_list.append(episode_reward)


    if episode < 100:
        avg_rew = np.mean(reward_list)
        print('\rEpisode {}\tAverage Score: {:.2f}'.format(episode, avg_rew), end="")
        avg_100 =  np.mean(reward_list)
        if avg_100 > -200.0:
          print('Stopped at Episode ',episode)
          n_episodes.append(episode)
          break
    if episode >= 100:
      avg_rew = np.mean(reward_list[-100:])
      print('\rEpisode {}\tAverage Score: {:.2f}'.format(episode, avg_rew), end="")
    if episode%100 == 0:
      avg_rew = np.mean(reward_list[-100:])
      print('\rEpisode {}\tAverage Score: {:.2f}'.format(episode, avg_rew))

    if episode >= 100:
      avg_100 =  np.mean(reward_list[-100:])
      if avg_100 > -200.0:
        print('Stopped at Episode ',episode)
        n_episodes.append(episode)
        break
  total_rewards.append(reward_list)
  time_taken = datetime.datetime.now() - begin_time
  print(time_taken)

Episode 10	Average Score: -425.40

In [ ]:
import seaborn as sns
def plot_mean_and_variance(mean_array, std_array):
    x = np.arange(len(mean_array))
    plt.plot(x, mean_array, label='Mean')
    plt.fill_between(x, mean_array - std_array, mean_array + std_array, alpha=0.2, label='Standard Deviation')
    sns.set_style('darkgrid')
    plt.xlabel('Episodes', fontsize = 16)
    plt.ylabel('Reward', fontsize = 16)